In [4]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

###  Overview 
To use this Jupyter notebook, copy the notebook to a Google Cloud Notebooks instance and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Jupyter automatically displays the return value of the last line in each cell. For more information about running notebooks in Google Cloud Notebook, see the Google Cloud Notebook guide.. 

### Objective 

This notebook demonstrate how to create a Custom Model using Custom Container Training and a Big Query Dataset. It will require you provide a bucket where the dataset will be stored.

Costs  
This tutorial uses billable components of Google Cloud: 

Vertex AI
Cloud Storage
Learn about Vertex AI  pricing and Cloud Storage pricing, and use the Pricing Calculator to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

# Ensure the following APIs are enabled:
- [BigQuery](https://console.cloud.google.com/apis/library/bigquery.googleapis.com?q=BigQuery)
- [Cloudbuild](https://console.cloud.google.com/apis/library/cloudbuild.googleapis.com?q=Cloudbuild)
- [Container Registry](https://console.cloud.google.com/apis/library/containerregistry.googleapis.com?q=container%20registry)

# Install Vertex AI SDK for Python


After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     |████████████████████████████████| 1.8 MB 15.4 MB/s 
     |████████████████████████████████| 107 kB 49.8 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 402 kB 58.6 MB/s 
     |████████████████████████████████| 107 kB 49.3 MB/s 
     |████████████████████████████████| 107 kB 60.9 MB/s 
     |████████████████████████████████| 106 kB 56.0 MB/s 
     |████████████████████████████████| 106 kB 69.7 MB/s 
     |████████████████████████████████| 106 kB 50.5 MB/s 
     |████████████████████████████████| 106 kB 54.7 MB/s 
     |████████████████████████████████| 105 kB 41.0 MB/s 
     |████████████████████████████████| 105 kB 55.4 MB/s 
     |████████████████████████████████| 105 kB 50.0 MB/s 
     |████████████████████████████████| 105 kB 52.1 MB/s 
     |████████████████████████████████| 105 kB 71.8 MB/s 
     |████████████████████████████████| 104 kB 69.5 MB/s 
     |████████████████████████████████| 104 kB 70.1 MB/s 
     |██████████

{'restart': True, 'status': 'ok'}

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
    
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

Project ID:  


Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"   # @param {type:"string"}
    print("Project ID: ", PROJECT_ID)

Project ID:  vertex-ai-dev


### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. We suggest that you [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions).

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://vertex-ai-devaip-20220428142746/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

# Copy Big Query Iris Dataset
We will make a Big Query dataset and copy Big Query's public iris table to that dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/iris 

### Make BQ Dataset

In [ ]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
!bq mk {PROJECT_ID}:ml_datasets

BigQuery error in mk operation: Dataset 'vertex-ai-dev:ml_datasets' already
exists.


### Copy bigquery-public-data.ml_datasets.iris

In [ ]:
!bq cp -n --project_id={PROJECT_ID} bigquery-public-data:ml_datasets.iris {PROJECT_ID}:ml_datasets.iris 

Table 'vertex-ai-dev:ml_datasets.iris' already exists, skipping


# Create Training Container
We will create a directory and write all of our container build artifacts into that folder.

In [ ]:
CONTAINER_ARTIFACTS_DIR = "demo-container-artifacts"

!mkdir {CONTAINER_ARTIFACTS_DIR}

### Create Cloudbuild YAML

In [ ]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/{PROJECT_ID}/test-custom-container', '.' ]
images: ['gcr.io/{PROJECT_ID}/test-custom-container']""".format(
    PROJECT_ID=PROJECT_ID
)

with open(f"{CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### Write The Dockerfile

In [ ]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/Dockerfile

# Specifies base image and tag
FROM gcr.io/google-appengine/python
WORKDIR /root

# Installs additional packages
RUN pip3 install tensorflow tensorflow-io pyarrow

# Copies the trainer code to the docker image.
COPY test_script.py /root/test_script.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "test_script.py"]

Writing demo-container-artifacts/Dockerfile


### Write the entrypoint script to invoke trainer

In [ ]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/test_script.py

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from tensorflow import feature_column
import os

training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
validation_data_uri = os.environ["AIP_VALIDATION_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]
data_format = os.environ["AIP_DATA_FORMAT"]

def caip_uri_to_fields(uri):
    uri = uri[5:]
    project, dataset, table = uri.split('.')
    return project, dataset, table

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

target_name = 'species'

def transform_row(row_dict):
  # Trim all string tensors
  trimmed_dict = { column:
                  (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                  for (column,tensor) in row_dict.items()
                  }
  target = trimmed_dict.pop(target_name)

  target_float = tf.cond(tf.equal(tf.strings.strip(target), 'versicolor'), 
                 lambda: tf.constant(1.0),
                 lambda: tf.constant(0.0))
  return (trimmed_dict, target_float)

def read_bigquery(project, dataset, table):
  tensorflow_io_bigquery_client = BigQueryClient()
  read_session = tensorflow_io_bigquery_client.read_session(
      "projects/" + project,
      project, table, dataset,
      feature_names + [target_name],
      [dtypes.float64] * 4 + [dtypes.string],
      requested_streams=2)

  dataset = read_session.parallel_read_rows()
  transformed_ds = dataset.map(transform_row)
  return transformed_ds

BATCH_SIZE = 16

training_ds = read_bigquery(*caip_uri_to_fields(training_data_uri)).shuffle(10).batch(BATCH_SIZE)
eval_ds = read_bigquery(*caip_uri_to_fields(validation_data_uri)).batch(BATCH_SIZE)
test_ds = read_bigquery(*caip_uri_to_fields(test_data_uri)).batch(BATCH_SIZE)

feature_columns = []

# numeric cols
for header in feature_names:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
    feature_layer,
      Dense(16, activation=tf.nn.relu),
      Dense(8, activation=tf.nn.relu),
      Dense(4, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid),
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'],
    optimizer='adam')

model.fit(training_ds, epochs=5, validation_data=eval_ds)

print(model.evaluate(test_ds))

tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Writing demo-container-artifacts/test_script.py


### Build The Container

In [ ]:
!gcloud builds submit --project={PROJECT_ID} --config {CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml {CONTAINER_ARTIFACTS_DIR}

Creating temporary tarball archive of 3 file(s) totalling 3.1 KiB before compression.
Uploading tarball of [demo-container-artifacts] to [gs://vertex-ai-dev_cloudbuild/source/1651156805.847043-f9151b3b01084e72ac8c41bc235a520e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/vertex-ai-dev/locations/global/builds/11e75ab2-4805-4aa0-a501-89a7121a5e3d].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/11e75ab2-4805-4aa0-a501-89a7121a5e3d?project=931647533046].
 REMOTE BUILD OUTPUT
starting build "11e75ab2-4805-4aa0-a501-89a7121a5e3d"

FETCHSOURCE
Fetching storage object: gs://vertex-ai-dev_cloudbuild/source/1651156805.847043-f9151b3b01084e72ac8c41bc235a520e.tgz#1651156806447310
Copying gs://vertex-ai-dev_cloudbuild/source/1651156805.847043-f9151b3b01084e72ac8c41bc235a520e.tgz#1651156806447310...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.
BUILD
Already have image (with dige

# Run The Custom Container Training

## Initialize The Vertex AI SDK for Python

Initialize the *client* for Vertex AI

In [3]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

ImportError: ignored

# Create a Managed Tabular Dataset from Big Query Dataset

This section will create a managed Tabular dataset from the iris Big Query table we copied above.

In [ ]:
ds = aiplatform.TabularDataset.create(
    display_name="bq_iris_dataset", bq_source=f"bq://{PROJECT_ID}.ml_datasets.iris"
)

Creating TabularDataset
Create TabularDataset backing LRO: projects/931647533046/locations/us-central1/datasets/8641682007249649664/operations/523981680784965632
TabularDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/8641682007249649664
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/931647533046/locations/us-central1/datasets/8641682007249649664')


# Launch The Training Job to Create a Model

We will train a model with the container we built above.

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="train-bq-iris",
    container_uri=f"gcr.io/{PROJECT_ID}/test-custom-container:latest",
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(
    ds,
    replica_count=1,
    model_display_name="bq-iris-model",
    bigquery_destination=f"bq://{PROJECT_ID}",
)

Training Output directory:
gs://vertex-ai-devaip-20220428142746/aiplatform-custom-training-2022-04-28-14:44:30.360 
No dataset split provided. The service will use a default split.
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1946107543617011712?project=931647533046
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/1946107543617011712 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/1946107543617011712 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/1946107543617011712 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/1946107543617011712 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/9316475

# Deploy The Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

Creating Endpoint
Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/46979932831612928/operations/2320917932105793536
Endpoint created. Resource name: projects/931647533046/locations/us-central1/endpoints/46979932831612928
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/931647533046/locations/us-central1/endpoints/46979932831612928')
Deploying model to Endpoint : projects/931647533046/locations/us-central1/endpoints/46979932831612928
Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/46979932831612928/operations/6932603950533181440
Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/46979932831612928


# Make a prediction


In [ ]:
endpoint.predict(
    [{"sepal_length": 5.1, "sepal_width": 2.5, "petal_length": 3.0, "petal_width": 1.1}]
)

Prediction(predictions=[[0.543015361]], deployed_model_id='8055171718668353536', explanations=None)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [2]:
import os

# Delete endpoint resource
! gcloud ai endpoints delete $ENDPOINT_NAME --quiet --region $REGION_NAME

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

if os.getenv("IS_TESTING"):
! gsutil -m rm -r $BUCKET_URI  